# Tarefa 2b: Resumo de textos abstrativo

Neste caderno, você gerencia os desafios decorrentes do resumo de documentos grandes. O texto de entrada pode exceder o tamanho do contexto do modelo, gerar saídas alucinadas ou acionar erros de falta de memória.

Para mitigar esses problemas, esse caderno demonstra uma arquitetura que usa fragmentação e encadeamento de prompts com o framework [LangChain](https://python.langchain.com/docs/get_started/introduction.html), um toolkit que permite que aplicações usem modelos de linguagem.

Você verá uma abordagem de cenários em que os documentos do usuário ultrapassam os limites de tokens. A fragmentação divide os documentos em segmentos menores do que os limites de comprimento do contexto antes de alimentá-los sequencialmente nos modelos. Isso encadeia prompts em blocos, mantendo o contexto anterior. Você aplica essa abordagem para resumir transcrições de chamadas, transcrições de reuniões, livros, artigos, publicações de blogs e outros conteúdos relevantes.

## Tarefa 2b.1: Configurar o ambiente

Nessa tarefa, você configura seu ambiente e cria um cliente Bedrock que detecta automaticamente sua região da AWS.

In [ ]:
#Create a service client by name using the default session.
import json
import os
import sys
import time
import random
from typing import Any, List, Mapping, Optional

# AWS and Bedrock imports
import boto3

# Get the region programmatically
session = boto3.session.Session()
region = session.region_name or "us-east-1"  # Default to us-east-1 if region not set

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime', region_name=region)


## Tarefa 2b.2: Resumir texto longo 

### Configurar o LangChain com o Boto3

Nessa tarefa, você especifica o LLM para a classe LangChain Bedrock e passa argumentos para inferência.

In [ ]:
# LangChain imports
from langchain_aws import BedrockLLM
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.chains.summarize import load_summarize_chain
from langchain_core.callbacks.manager import CallbackManagerForLLMRun
from langchain_core.language_models.llms import LLM

# Base LLM configuration
modelId = "amazon.nova-lite-v1:0"

class NovaLiteWrapper(LLM):
    """Wrapper for Nova Lite model that formats inputs correctly."""
    
    @property
    def _llm_type(self) -> str:
        return "nova-lite-wrapper"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Format prompt for Nova Lite and process."""
        # Format the prompt for Nova Lite's expected message structure
        formatted_input = {
            "messages": [
                {
                    "role": "user",
                    "content": [{"text": prompt}]  # Content must be an array with text objects
                }
            ],
            "inferenceConfig": {
                "maxTokens": 2048,
                "temperature": 0,
                "topP": 0.9
            }
        }
        
        # Call Bedrock directly with the properly formatted input
        response = bedrock_client.invoke_model(
            modelId=modelId,
            body=json.dumps(formatted_input)
        )
        
        # Parse the response - updated to handle Nova Lite's response format
        response_body = json.loads(response['body'].read().decode('utf-8'))
        
        # Extract the text from the response
        if 'output' in response_body and 'message' in response_body['output']:
            message = response_body['output']['message']
            if 'content' in message and isinstance(message['content'], list):
                # Extract text from each content item
                texts = []
                for content_item in message['content']:
                    if isinstance(content_item, dict) and 'text' in content_item:
                        texts.append(content_item['text'])
                return ' '.join(texts)
        
        # Fallback if the response format is different
        return str(response_body)
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {"model_id": modelId}
    
    def get_num_tokens(self, text: str) -> int:
        """Estimate token count - Nova Lite uses roughly 1 token per 4 characters."""
        return len(text) // 4  # Rough approximation

# Create the Nova Lite wrapper
llm = NovaLiteWrapper()


## Criação de wrapper de LLM otimizado para recursos

Para lidar com as cotas de serviço do Bedrock de forma eficaz, criaremos uma classe wrapper que otimiza o uso de recursos e implementa um recuo exponencial com jitter para chamadas de API.

In [ ]:
# Enhanced resource-optimized LLM wrapper with exponential backoff
class ResourceOptimizedLLM(LLM):
    """Wrapper that optimizes resource usage for LLM processing."""
    
    llm: Any  # The base LLM to wrap
    min_pause: float = 30.0  # Minimum pause between requests
    max_pause: float = 60.0  # Maximum pause after throttling
    initial_pause: float = 10.0  # Initial pause between requests
    
    @property
    def _llm_type(self) -> str:
        return f"optimized-{self.llm._llm_type}"
    
    def _call(
        self,
        prompt: str,
        stop: Optional[List[str]] = None,
        run_manager: Optional[CallbackManagerForLLMRun] = None,
        **kwargs: Any,
    ) -> str:
        """Process with resource optimization and exponential backoff."""
        # Always pause between requests to optimize resource usage
        time.sleep(self.initial_pause)
        
        # Implement retry with exponential backoff
        max_retries = 10  # More retries for important operations
        base_delay = self.min_pause
        
        for attempt in range(max_retries):
            try:
                print(f"Making API call (attempt {attempt+1}/{max_retries})...")
                return self.llm._call(prompt, stop=stop, run_manager=run_manager, **kwargs)
            
            except Exception as e:
                error_str = str(e)
                
                # Handle different types of service exceptions
                if any(err in error_str for err in ["ThrottlingException", "TooManyRequests", "Rate exceeded"]):
                    if attempt < max_retries - 1:
                        # Calculate backoff with jitter to prevent request clustering
                        jitter = random.random() * 0.5
                        wait_time = min(base_delay * (2 ** attempt) + jitter, self.max_pause)
                        
                        print(f"Service capacity reached. Backing off for {wait_time:.2f} seconds...")
                        time.sleep(wait_time)
                    else:
                        print("Maximum retries reached. Consider reducing batch size or increasing delays.")
                        raise
                else:
                    # For non-capacity errors, don't retry
                    print(f"Non-capacity error: {error_str}")
                    raise
    
    @property
    def _identifying_params(self) -> Mapping[str, Any]:
        return {**self.llm._identifying_params, "initial_pause": self.initial_pause}
    
    def get_num_tokens(self, text: str) -> int:
        """Pass through token counting to the base model."""
        return self.llm.get_num_tokens(text)

# Create the resource-optimized LLM
resource_optimized_llm = ResourceOptimizedLLM(llm=llm, initial_pause=10.0)



<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** este wrapper adiciona recursos importantes para uso em produção:

- Pausa automática entre solicitações para respeitar as cotas de serviço
- Recuo exponencial com jitter para lidar com exceções de controle de utilização
- Tratamento e geração de relatórios abrangentes de erros

## Tarefa 2b.3: Carregar arquivo de texto com muitos tokens

Nessa tarefa, você usa uma cópia da [carta do CEO da Amazon aos acionistas em 2022](https://www.aboutamazon.com/news/company-news/amazon-ceo-andy-jassy-2022-letter-to-shareholders) no diretório de cartas. Você cria uma função para carregar o arquivo de texto e lidar com possíveis erros.

In [ ]:
# Document loading function
def load_document(file_path):
    """Load document from file."""
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            content = file.read()
        return content
    except Exception as e:
        print(f"Error loading document: {e}")
        return None

# Example usage
shareholder_letter = "../letters/2022-letter.txt"
letter = load_document(shareholder_letter)

if letter:
    num_tokens = resource_optimized_llm.get_num_tokens(letter)
    print(f"Document loaded successfully with {num_tokens} tokens")


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** você pode ignorar os avisos com segurança e prosseguir para a próxima célula. Resolveremos isso fragmentando o documento na próxima etapa.

## Tarefa 2b.4: Dividir o texto longo em blocos

Nessa tarefa, você divide o texto em partes menores porque ele é muito longo para caber no prompt. `RecursiveCharacterTextSplitter` no LangChain suporta a divisão de texto longo em partes recursivamente até que o tamanho de cada bloco se torne menor que chunk_size.

In [ ]:
# Document chunking with conservative settings
def chunk_document(text, chunk_size=4000, chunk_overlap=200):
    """Split document into manageable chunks."""
    text_splitter = RecursiveCharacterTextSplitter(
        separators=["\n\n", "\n", ".", " "],
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    
    chunks = text_splitter.create_documents([text])
    print(f"Document split into {len(chunks)} chunks")
    return chunks

# Split the document into chunks
if letter:
    docs = chunk_document(letter, chunk_size=4000, chunk_overlap=200)
    
    if docs:
        num_docs = len(docs)
        num_tokens_first_doc = resource_optimized_llm.get_num_tokens(docs[0].page_content)
        print(f"Now we have {num_docs} documents and the first one has {num_tokens_first_doc} tokens")


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** o parâmetro `chunk_size` controla o tamanho de cada bloco. Blocos maiores fornecem mais contexto, mas exigem mais recursos de processamento. O parâmetro `chunk_overlap` garante alguma continuidade entre os blocos.

## Tarefa 2b.5: Resumir e reunir blocos

Nessa tarefa, você implementa duas abordagens para resumir documentos fragmentados: usando a cadeia de resumo integrada do LangChain e uma implementação manual personalizada que fornece melhor controle sobre o uso de recursos.

## Entender abordagens de implementação

Este caderno demonstra duas abordagens diferentes para resumir documentos grandes com o AWS Bedrock:

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** incluímos uma implementação padrão do LangChain e uma implementação personalizada para mostrar as vantagens e desvantagens entre conveniência e controle ao criar aplicativos de produção.

### Dois caminhos para o mesmo objetivo

1. **Implementação padrão do LangChain** (`process_documents_with_pacing`):
   - Usa as cadeias de resumo integradas do LangChain
   - Requer menos código e é mais fácil de implementar
   - Abstrai a complexidade subjacente
   - Ótimo para prototipagem rápida e casos de uso simples

2. **Implementação de refinamento personalizada** (`manual_refine_with_optimization`):
   - Constrói o processo de refinamento passo a passo
   - Fornece visibilidade completa dos prompts e do processamento
   - Oferece tratamento granular de erros para cada bloco do documento
   - Permite um controle preciso sobre o tempo de chamada da API e a lógica de repetição

Embora ambas alcancem o mesmo resultado final, a implementação personalizada oferece mais controle sobre todo o processo, o que é crucial ao trabalhar com cotas de serviço e criar aplicativos prontos para produção.

Em cenários reais, você pode começar com a implementação padrão durante o desenvolvimento e depois passar para uma implementação personalizada quando precisar de mais controle sobre o uso de recursos, tratamento de erros ou engenharia de prompts.

### Implementação padrão do LangChain

In [ ]:
# Custom document processing with controlled pacing
def process_documents_with_pacing(docs, chain_type="refine", verbose=True):
    """Process documents with pacing to optimize resource usage."""
    
    # Configure the chain
    summary_chain = load_summarize_chain(
        llm=resource_optimized_llm,
        chain_type=chain_type,  # "refine" processes sequentially, good for resource optimization
        verbose=verbose
    )
    
    # Process with additional error handling
    try:
        result = summary_chain.invoke(docs)
        return result
    except ValueError as error:
        if "AccessDeniedException" in str(error):
            print(f"\n\033[91mAccess Denied: {error}\033[0m")
            print("\nTo troubleshoot this issue, please check:")
            print("1. Your IAM permissions for Bedrock")
            print("2. Model access permissions")
            print("3. AWS credentials configuration")
            return {"output_text": "Error: Access denied. Check permissions."}
        else:
            print(f"\n\033[91mError during processing: {error}\033[0m")
            return {"output_text": f"Error during processing: {str(error)}"}


### Implementação personalizada de refinamento com otimização aprimorada de recursos

In [ ]:
# Manual implementation of resource-optimized processing for refine chain
def manual_refine_with_optimization(docs, llm, verbose=True):
    """Manually implement refine chain with resource optimization."""
    if not docs:
        return {"output_text": "No documents to process."}
    
    # Process first document to get initial summary
    print(f"Processing initial document (1/{len(docs)})...")
    
    # Simple prompt for initial document
    initial_prompt = """Write a concise summary of the following:
    "{text}"
    CONCISE SUMMARY:"""
    
    # Process first document
    try:
        current_summary = llm(initial_prompt.format(text=docs[0].page_content))
        print("Initial summary created successfully.")
    except Exception as e:
        print(f"Error creating initial summary: {e}")
        return {"output_text": "Failed to create initial summary."}
    
    # Process remaining documents with refine approach
    for i, doc in enumerate(docs[1:], start=2):
        print(f"Refining with document {i}/{len(docs)}...")
        
        # Refine prompt
        refine_prompt = """Your job is to refine an existing summary.
        We have an existing summary: {existing_summary}
        
        We have a new document to add information from: {text}
        
        Please update the summary to incorporate new information from the document.
        If the document doesn't contain relevant information, return the existing summary.
        
        REFINED SUMMARY:"""
        
        try:
            # Apply resource optimization between requests
            time.sleep(10.0)  # Base delay between requests
            
            # Update the summary
            current_summary = llm(refine_prompt.format(
                existing_summary=current_summary,
                text=doc.page_content
            ))
            
            if verbose:
                print(f"Successfully refined with document {i}")
        except Exception as e:
            print(f"Error during refinement with document {i}: {e}")
            # Apply exponential backoff
            backoff = min(10.0 * (2 ** (i % 5)) + (random.random() * 2), 30)
            print(f"Backing off for {backoff:.2f} seconds...")
            time.sleep(backoff)
            
            # Try one more time
            try:
                current_summary = llm(refine_prompt.format(
                    existing_summary=current_summary,
                    text=doc.page_content
                ))
            except Exception as retry_error:
                print(f"Retry failed for document {i}: {retry_error}")
                # Continue with current summary rather than failing completely
    
    return {"output_text": current_summary}


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** a implementação manual oferece mais controle sobre:

- Os prompts exatos usados para resumir
- Tratamento de erros e recuperação
- Otimização de recursos entre chamadas de API
- Degradação graciosa quando ocorrem erros

## Tarefa 2b.6: Função de execução principal

Agora, criaremos uma função principal que orquestra todo o processo de resumo do documento.

<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** a função principal (`summarize_document`) permite que você escolha qual implementação usar com base no parâmetro `chain_type`, facilitando a comparação de resultados e desempenho.

In [ ]:
# Main execution function
def summarize_document(file_path, chunk_size=4000, chain_type="refine"):
    """Main function to summarize a document."""
    
    print(f"Starting document summarization process for: {file_path}")
    
    # Load the document
    document_text = load_document(file_path)
    if not document_text:
        return "Failed to load document."
    
    print(f"Document loaded successfully. Length: {len(document_text)} characters")
    
    # Split into chunks
    docs = chunk_document(document_text, chunk_size=chunk_size, chunk_overlap=200)
    
    # If document is very large, provide a warning
    if len(docs) > 15:
        print(f"Warning: Document is large ({len(docs)} chunks). Processing may take some time.")
        
        # For very large documents, consider using a subset for testing
        if len(docs) > 30:
            print("Document is extremely large. Consider using a smaller chunk_size or processing a subset.")
            # Optional: process only a subset for testing
            # docs = docs[:15]
    
    # Process the documents
    print(f"Processing document using '{chain_type}' chain type...")
    
    # Use the appropriate processing method based on chain type
    if chain_type == "refine":
        # Use our manual implementation for better control over resource optimization
        result = manual_refine_with_optimization(docs, resource_optimized_llm)
    else:
        # Use standard LangChain implementation for other chain types
        result = process_documents_with_pacing(docs, chain_type=chain_type)
    
    # Return the result
    if result and "output_text" in result:
        print("\nSummarization completed successfully!")
        return result["output_text"]
    else:
        print("\nSummarization failed or returned no result.")
        return "Summarization process did not produce a valid result."


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i>**Nota: ** Dependendo do número de documentos, da cota da taxa de solicitações do Bedrock e das configurações de repetição definidas, o processo de resumo pode levar algum tempo para ser executado.

## Tarefa 2b.7: Executar o resumo

Vamos fazer um resumo da carta para os acionistas. Por padrão, a função summarize_document() usa a cadeia de refinamento. Para ativar o map_reduce: 

- Incluir comentário na seguinte linha: `summary = summarize_document(document_path, chunk_size=4000, chain_type="refine")`
- Remover o comentário da seguinte linha: `# summary = summarize_document(document_path, chunk_size=4000, chain_type="map_reduce")`


<i aria-hidden="true" class="fas fa-sticky-note" style="color:#563377"></i> **Nota:** não se preocupe se você notar mensagens de erro durante a execução. Seu código inclui um tratamento robusto de erros que repetirá automaticamente as solicitações que falharem com recuo exponencial. Esse é um comportamento normal ao trabalhar com cotas de serviço e demonstra como os aplicativos prontos para produção devem lidar com as limitações da API.

In [ ]:
# Example usage
if __name__ == "__main__":
    # Path to your document
    document_path = "../letters/2022-letter.txt"
    
    # Summarize with different options
    # Option 1: Standard refine chain (sequential processing, good for resource optimization)
    summary = summarize_document(document_path, chunk_size=4000, chain_type="refine")
    
    # Option 2: For comparison, you could try map_reduce (but be careful with service quotas)
    # summary = summarize_document(document_path, chunk_size=4000, chain_type="map_reduce")
    
    # Print the final summary
    print("\n=== FINAL SUMMARY ===\n")
    print(summary)


Você testou o uso da fragmentação e do encadeamento de prompts com o framework LangChain para resumir documentos grandes e, ao mesmo tempo, mitigar problemas decorrentes de textos longos de entrada.

## Para entender os principais componentes

Vamos analisar os principais componentes da nossa solução:

1. **Otimização de recursos**: o wrapper `ResourceOptimizedLLM` gerencia as chamadas de API para permanecer dentro das cotas de serviço do Bedrock:
   - Adicionando pausas entre solicitações (controlado por `initial_pause`)
   - Implementando recuo exponencial com jitter quando ocorre controle de utilização
   - Fornecendo tratamento e recuperação abrangentes de erros

2. **Fragmentação de documentos**: a função `chunk_document` divide documentos grandes em partes gerenciáveis:
   - `chunk_size` controla o tamanho máximo de cada bloco (4.000 caracteres)
   - `chunk_overlap` garante a continuidade do contexto entre os blocos (200 caracteres)
   - Separadores de texto natural (`\n\n`, `\n`, `.` etc.) são usados para evitar quebrar o meio do parágrafo

3. **Abordagens de resumo**:
   - **Refinar a cadeia**: processa os blocos sequencialmente, refinando o resumo com cada novo bloco
   - **Map-Reduce**: resume cada bloco de forma independente e, em seguida, reúne e resume esses resumos

4. **Tratamento de erros**: o tratamento abrangente de erros garante que o processo possa se recuperar de:
   - Controle de utilização do serviço e limitação da capacidade
   - Problemas de permissão de acesso
   - Outros erros de API

## Experimente você mesmo

- Altere os prompts para seu caso de uso específico e avalie o resultado de diferentes modelos.
- Experimente diferentes tamanhos de blocos para encontrar o equilíbrio ideal entre preservação de contexto e eficiência de processamento.
- Experimente diferentes tipos de cadeia de resumo (`refine` versus `map_reduce`) e compare os resultados.
- Ajuste os parâmetros de otimização de recursos com base nos limites de cota do Bedrock.

### Aplicações práticas

Essa abordagem pode ser aplicada para resumir vários tipos de conteúdo longo:
- Transcrições de chamadas de atendimento ao cliente
- Transcrições e notas de reuniões
- Artigos de pesquisa e documentos técnicos
- Documentos jurídicos e contratos
- Livros, artigos e postagens em blogs

### Práticas recomendadas

Ao implementar essa solução na produção:

1. **Monitore o uso da API**: acompanhe suas chamadas de API para ficar dentro dos limites da cota
2. **Otimize tamanho do bloco**: equilíbrio entre preservação de contexto e eficiência de processamento
3. **Implemente o tratamento adequado de erros**: garanta que seu aplicativo possa lidar corretamente com os erros da API
4. **Considere o armazenamento em cache**: armazene os resultados em cache para evitar chamadas de API redundantes para documentos acessados com frequência
5. **Teste vários tipos de documentos**: conteúdos diferentes podem exigir diferentes estratégias de fragmentação

### Limpeza

Você concluiu este caderno. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de caderno e continue com a **Tarefa 3**.